In [21]:
import numpy as np
import pandas as pd
import random
import os

In [22]:
data_path = '/content/drive/MyDrive/BKT/skill_builder_data.csv'

In [23]:
data = pd.read_csv(data_path, nrows=50000 ,encoding='latin-1')
data_frame = pd.DataFrame(data)

In [24]:
data_frame.drop(np.where(data_frame['skill_id'].isna())[0],axis=0, inplace=True)

In [25]:
len(data),len(data_frame)

(50000, 50000)

In [26]:
v = data_frame.user_id.value_counts()

In [27]:
data_frame = data_frame[data_frame.user_id.isin(v.index[v.gt(3)])]
print(len(data_frame))

49468


In [28]:
print(len(pd.unique(data_frame['skill_id'])))
print(len(pd.unique(data_frame['user_id'])))

10
1213


In [29]:
len(set(data_frame["problem_id"]))

2365

In [30]:
data_frame.reset_index(inplace=True)

In [31]:
uid_pid = {}
user = data_frame["user_id"]
problem = data_frame["problem_id"]
for i in range(len(user)):
  # print(user,end=",")
  if user[i] in uid_pid:
    uid_pid[user[i]].append(problem[i])
  if user[i] not in uid_pid:
    uid_pid[user[i]] = [problem[i]]
    

In [32]:
pid_sid = {}
problem = data_frame["problem_id"]
skill = data_frame["skill_id"]
for i in range(len(problem)):
  # print(user,end=",")
  if problem[i] in pid_sid:
    pid_sid[problem[i]].append(skill[i])
  if problem[i] not in pid_sid:
    pid_sid[problem[i]]=[skill[i]]

In [33]:
sid_response = {}
skill = data_frame["skill_id"]
resp = data_frame['correct']
for i in range(len(skill)):
  if skill[i] in sid_response:
    sid_response[skill[i]].append(resp[i])
  if skill[i] not in sid_response:
    sid_response[skill[i]] = [resp[i]]


In [35]:
from tqdm import tqdm

In [36]:
user_skill = {}

u = list(uid_pid.keys())
print(len(uid_pid[u[0]]))
for i in tqdm(range(len(u))):
  pid = uid_pid[u[i]]
  skills_probabilities = {}
  for j in range(len(pid)):
    sid = pid_sid[pid[j]]
    for k in range(len(sid)):
      p_init = 0.5
      if sid[k] in skills_probabilities:
        p_init = skills_probabilities[sid[k]]
      p_guess = 0.3
      p_slip = 0.1
      p_learn = p_init
      resp_list = sid_response[sid[k]]

      for l in range(len(resp_list)):
        if resp_list[l] == 1:
          p_learn = (p_init*(1 - p_slip))/((p_init)*(1 - p_slip) + (1 - p_init)*p_guess)
        else:
          p_learn = (p_init * p_slip)/(p_init * p_slip + p_init * (1 - p_guess))
        p_init = p_learn
      
      skills_probabilities[sid[k]] = p_learn
  user_skill[u[i]] = skills_probabilities    

      


  0%|          | 0/1213 [00:00<?, ?it/s]

57



100%|██████████| 1213/1213 [9:18:34<00:00, 27.63s/it]


In [42]:
with open("/content/drive/MyDrive/bkt_probabilities.txt", 'w') as f:
	for key, value in user_skill.items():
		f.write('%s:%s\n' % (key, value))
